#  Optimization of Kv4.3 Wild Type

## Method: Genetic Algorithm

## Luca Sagresti

## 15/12/2019

In [4]:
%clear
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.integrate import odeint
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from scipy.optimize import least_squares

%matplotlib inline

mpl.rcParams['figure.dpi']=100
mpl.rcParams['figure.titlesize']=20
mpl.rcParams['axes.facecolor']='white'        
mpl.rcParams['lines.linewidth']=2.0
mpl.rcParams['axes.linewidth']=2.0
mpl.rcParams['xtick.major.pad']=6
mpl.rcParams['ytick.major.pad']=6
mpl.rcParams['xtick.labelsize']=14
mpl.rcParams['ytick.labelsize']=14
mpl.rcParams['axes.titlesize']=18
mpl.rcParams['axes.labelsize']=18
mpl.rcParams['axes.grid']='True'
mpl.rcParams['axes.axisbelow']='line'
mpl.rcParams['legend.fontsize']=12

## Import Experimental Data

In [5]:
import pandas as pd

folder="~/Downloads/Data_exp_Kv/"

# Activation dataset

dataset_act = pd.read_csv(folder+"Act_datasets.csv", skiprows=1)

act_data = dataset_act.to_numpy()

act_data_WT = act_data[:,1]

act_data_S390N = act_data[:,3]

# Inactivation dataset

dataset_inact = pd.read_csv(folder+"Inact_datasets.csv", skiprows=1)

inact_data = dataset_inact.to_numpy()

inact_data_WT = inact_data[:,1]

inact_data_S390N = inact_data[:,3]

# Recovery dataset

dataset_rec = pd.read_csv(folder+"Rec_datasets.csv", skiprows=1)

rec_data = dataset_rec.to_numpy()

rec_data_WT = rec_data[:,1]

rec_data_S390N = rec_data[:,3]



## Activation Part

In [6]:
def ode_act_WT (S, t, p):
    
    C0=S[0]
    C1=S[1]
    C2=S[2]
    C3=S[3]
    C4=S[4]
    I0=S[5]
    I1=S[6]
    I2=S[7]
    I3=S[8]
    I4=S[9]
    O =S[10]

    
    #constants
    
    T = 291.0 #K or 18 degree celsius
    e =  1.602176634 * (10**-19.0) # C
    K_B = 1.380649 * (10**-23.0) # J*K^-1
    
    exp_factor = (e/(K_B * T)) * (10**-3) 

    #Voltage sequences
    
    V = 0.0 #mV 

    if 0 <= t < 0.5:
        V = -90.0 # mV
        
    if 0.5 <= t < 0.55:
        V = p[11] # Vtest
        
    if 0.55 <= t <= 1.05:
        V = -50.0 # mV
        
    if t > 1.05 :
        V = -50.0 #mV
 


    # wild type parameters
#    alpha_0 = 450.0 #s^-1
#    alpha_1 = 0.23 #s^-1
    
#    beta_0 = 2.0 #s^-1
#    beta_1 = 2.2 #s^-1
    
#    k_CO_0 = 160.0 #s^-1
#    k_CO_1 = 0.27 #s^-1
    
#    k_OC_0 = 245.0 #s^-1
#    k_OC_1 = 0.33 #s^-1
    
#    k_CI = 25.0 #s^-1
#    k_IC = 0.3 #s^-1
    
#    f = 0.37

    k_CI = p[8]
    
    k_IC = p[9]

    f = p[10]
    
    #voltage dependent rate constants
    
    #alpha = alpha_0 * np.exp(alpha_1 * (V * exp_factor))
    #beta = beta_0 * np.exp(-1.0 * beta_1 * (V * exp_factor))
    #k_CO = k_CO_0 * np.exp(k_CO_1 * (V * exp_factor))
    #k_OC = k_OC_0 * np.exp(-1.0 * k_OC_1 * (V * exp_factor))
    
     
    alpha = p[0] * np.exp(p[1] * (V * exp_factor))
    beta = p[2] * np.exp(-1.0 * p[3] * (V * exp_factor))
    k_CO = p[4] * np.exp(p[5] * (V * exp_factor))
    k_OC = p[6] * np.exp(-1.0 * p[7] * (V * exp_factor))
    
    # ODEs
    
    dC0dt = beta * C1 + (k_IC/(f**4.0)) * I0 - (k_CI*(f**4.0) + 4.0 * alpha) * C0
    dC1dt = 4.0 * alpha * C0 + 2.0 * beta * C2 + (k_IC/(f**3.0)) * I1 - (k_CI*(f**3.0) + beta + 3.0 * alpha) * C1 
    dC2dt = 3.0 * alpha * C1 + 3.0 * beta * C3 + (k_IC/(f**2.0)) * I2 - (k_CI*(f**2.0) + 2.0 * beta + 2.0 * alpha) * C2 
    dC3dt = 2.0 * alpha * C2 + 4.0 * beta * C4 + (k_IC/f) * I3 - (k_CI*f + 3.0 * beta + 1.0 * alpha) * C3 
    dC4dt = 1.0 * alpha * C3 + k_OC * O + k_IC * I4 - (k_CI + k_CO + 4.0 * beta) * C4 
    
    dI0dt = beta * f * I1 + (k_CI*(f**4.0)) * C0 - (k_IC/(f**4.0) + 4.0 * (alpha/f)) * I0
    dI1dt = 4.0 * (alpha/f) * I0 + 2.0 * beta * f * I2 + (k_CI*(f**3.0)) * C1 - (k_IC/(f**3.0) + beta * f + 3.0 * (alpha/f)) * I1 
    dI2dt = 3.0 * (alpha/f) * I1 + 3.0 * beta * f * I3 + (k_CI*(f**2.0)) * C2 - (k_IC/(f**2.0) + 2.0 * beta * f + 2.0 * (alpha/f)) * I2 
    dI3dt = 2.0 * (alpha/f) * I2 + 4.0 * beta * f * I4 + (k_CI*f) * C3 - (k_IC/f + 3.0 * beta * f + 1.0 * (alpha/f)) * I3 
    dI4dt = 1.0 * (alpha/f) * I3 + k_CI * C4 - (k_IC + 4.0 * beta * f) * I4     
    
    dOdt = k_CO * C4 - k_OC * O
    
    return (dC0dt, dC1dt, dC2dt, dC3dt, dC4dt, dI0dt, dI1dt, dI2dt, dI3dt, dI4dt, dOdt)

In [7]:
def Act_Protocol(max_V, DeltaV):
    Vhold = -90.0 #mV
    
    Vtest = np.linspace(Vhold,max_V,np.abs(np.int((max_V-Vhold)/DeltaV))+1)
    
    return Vtest #array of testing voltages

In [8]:
def Boltz_eq(x, V1_2, k):
    return 1.0/(1.0 + np.exp(-(x-V1_2)/k))

In [9]:
def Act_LS_func(x, teta):

    # conductance parameters

    #EK      = 0.0    # mV
    gK_max  = 33.2     # nS 

    # Assuming no leaking
    #EL      = 0.0    # mV
    #gL_max  = 0.0    # mS 

    # Membrane capacitance
    Cm      = 1.0    # microF cm^-2
    
    #define activation sequence protocol
    
    Vmax = 60.0 #mV
    
    increment = 10.0 #mV
    
    Vtest = Act_Protocol(Vmax, increment)
    
    # Time discretiztion
    tini = 0.0
    
    tend = 1.05
    
    ttest_i = 0.50 #time at which you start record the current
    
    ttest_f = 0.55 #time at which you end record the current
     
    Npoints = 200000
    
    Points_per_sec = np.int(Npoints/tend) 
    
    # time array
    #t = np.linspace(tini,tend,Npoints)

    # prepare empty arrays
    Open_states = np.zeros((Npoints,len(Vtest)))

    max_conductance = np.zeros(len(Vtest)) 

    
    for i in range (0,len(Vtest)):
        gamma = np.append(teta, Vtest[i])
        
        f = lambda S,t: ode_act_WT(S, t, gamma)
        
        r = odeint(f, S0, x)
        
        Open_states[:,i] = r[:,10]
        
        max_conductance[i] = gK_max * np.amax(r[np.int(Points_per_sec*ttest_i):np.int(Points_per_sec*ttest_f),10])
    
    
    g_gmax = (max_conductance)/(np.amax(max_conductance))
    
    return g_gmax

In [10]:
def residual_Act(p):
    
    #define activation sequence protocol
    
    Vmax = 60.0 #mV
    
    increment = 10.0 #mV
    
    Vtest = Act_Protocol(Vmax, increment)
    
    #experimental data for hemiactivation and slope
    
    exp_Vhemi_act = -22.70 #mV
    
    exp_k_act = 13.00 #mV
    
    #evaluation of experimental points trough the declared Boltzmann fit
    
    exp_data = Boltz_eq(Vtest,exp_Vhemi_act,exp_k_act)
    
    # Time discretiztion
    tini = 0.0
    
    tend = 1.05
    
    ttest = 0.56 #time at which you start record the current
     
    Npoints = 200000
    
    Points_per_sec = np.int(Npoints/tend) 
    
    # time array
    t = np.linspace(tini,tend,Npoints)
    
    # here data from perfect fit
    sq_err_act = np.sum(np.subtract(exp_data,Act_LS_func(t,p))**2)
    
    #here experimental data imported with Pandas in initial section
    #sq_err_act = np.sum(np.subtract(act_data_WT,Act_LS_func(t,p))**2)
    
    act_cost_func = (1.0/len(Vtest)) * sq_err_act
    
    return act_cost_func

### Debugger Activation

In [90]:
# Main 

from scipy.optimize import minimize

# Guess of initial conditions for the states 
C0_0=0.4390
C1_0=0.2588
C2_0=0.0572
C3_0=0.0056
C4_0=0.0002
I0_0=0.0128
I1_0=0.0553
I2_0=0.0894
I3_0=0.0642
I4_0=0.0172
O_0=0.0001

# Pack up the initial conditions:

S0 = [C0_0, C1_0, C2_0, C3_0, C4_0, I0_0, I1_0, I2_0, I3_0, I4_0, O_0]

# guess of the transition rates and other ODEs params (start from Lin Model)

guess = np.array([690.87, 0.252, 45.32, 2.22, 326.15, 0.369, 65.48, 0.28, 65.0, 0.21, 0.32])

# set boundaries for parameters lsq fitting based on physical reasoning

boundaries = ((400.0,1000.0),(0.0,1.0),(0.0,100.0),(0.0,10.0),(0.0,1000.0),(0.0,1.0),(0.0,100.0),(0.0,1.0),(0.0,100.0),(0.0,1.0),(0,0.5))

# Run the optimization for the cost function 

#result_lm = least_squares(residual_Act, guess, method='lm')

#result = minimize(residual_Act, guess, bounds=boundaries, method='L-BFGS-B')
result = minimize(residual_Act, guess, method='Nelder-Mead')
print (result)

KeyboardInterrupt: 

## Inactivation Part

In [11]:
def ode_inac_WT(C, t, p):
    C0=C[0]
    C1=C[1]
    C2=C[2]
    C3=C[3]
    C4=C[4]
    I0=C[5]
    I1=C[6]
    I2=C[7]
    I3=C[8]
    I4=C[9]
    O=C[10]

    
    #constants
    
    T = 291.0 #K or 18 degree celsius
    e =  1.602176634 * (10**-19.0) # C
    K_B = 1.380649 * (10**-23.0) # J*K^-1
    
    exp_factor = (e/(K_B * T)) * (10**-3) 

    #Voltage sequences
    
    V = 0.0 #mV 

    if 0 <= t < 1.00:
        V=-90.0
        
    if 1.00 <= t < 2.00:
        V=p[11]
        
    if 2.00 <= t <= 3.00:
        V=60.0
        
    if t > 3.00 :
        V=60.0
 


    # wild type parameters
#    alpha_0 = 450.0 #s^-1
#    alpha_1 = 0.23 #s^-1
    
#    beta_0 = 2.0 #s^-1
#    beta_1 = 2.2 #s^-1
    
#    k_CO_0 = 160.0 #s^-1
#    k_CO_1 = 0.27 #s^-1
    
#    k_OC_0 = 245.0 #s^-1
#    k_OC_1 = 0.33 #s^-1
    
#    k_CI = 25.0 #s^-1
#    k_IC = 0.3 #s^-1
    
#    f = 0.37

    k_CI = p[8]
    
    k_IC = p[9]

    f = p[10]
    
    #voltage dependent rate constants
    
    #alpha = alpha_0 * np.exp(alpha_1 * (V * exp_factor))
    #beta = beta_0 * np.exp(-1.0 * beta_1 * (V * exp_factor))
    #k_CO = k_CO_0 * np.exp(k_CO_1 * (V * exp_factor))
    #k_OC = k_OC_0 * np.exp(-1.0 * k_OC_1 * (V * exp_factor))
    
     
    alpha = p[0] * np.exp(p[1] * (V * exp_factor))
    beta = p[2] * np.exp(-1.0 * p[3] * (V * exp_factor))
    k_CO = p[4] * np.exp(p[5] * (V * exp_factor))
    k_OC = p[6] * np.exp(-1.0 * p[7] * (V * exp_factor))
    
    # ODEs
    
    dC0dt = beta * C1 + (k_IC/(f**4.0)) * I0 - (k_CI*(f**4.0) + 4.0 * alpha) * C0
    dC1dt = 4.0 * alpha * C0 + 2.0 * beta * C2 + (k_IC/(f**3.0)) * I1 - (k_CI*(f**3.0) + beta + 3.0 * alpha) * C1 
    dC2dt = 3.0 * alpha * C1 + 3.0 * beta * C3 + (k_IC/(f**2.0)) * I2 - (k_CI*(f**2.0) + 2.0 * beta + 2.0 * alpha) * C2 
    dC3dt = 2.0 * alpha * C2 + 4.0 * beta * C4 + (k_IC/f) * I3 - (k_CI*f + 3.0 * beta + 1.0 * alpha) * C3 
    dC4dt = 1.0 * alpha * C3 + k_OC * O + k_IC * I4 - (k_CI + k_CO + 4.0 * beta) * C4 
    
    dI0dt = beta * f * I1 + (k_CI*(f**4.0)) * C0 - (k_IC/(f**4.0) + 4.0 * (alpha/f)) * I0
    dI1dt = 4.0 * (alpha/f) * I0 + 2.0 * beta * f * I2 + (k_CI*(f**3.0)) * C1 - (k_IC/(f**3.0) + beta * f + 3.0 * (alpha/f)) * I1 
    dI2dt = 3.0 * (alpha/f) * I1 + 3.0 * beta * f * I3 + (k_CI*(f**2.0)) * C2 - (k_IC/(f**2.0) + 2.0 * beta * f + 2.0 * (alpha/f)) * I2 
    dI3dt = 2.0 * (alpha/f) * I2 + 4.0 * beta * f * I4 + (k_CI*f) * C3 - (k_IC/f + 3.0 * beta * f + 1.0 * (alpha/f)) * I3 
    dI4dt = 1.0 * (alpha/f) * I3 + k_CI * C4 - (k_IC + 4.0 * beta * f) * I4     
    
    dOdt = k_CO * C4 - k_OC * O
    
    return (dC0dt, dC1dt, dC2dt, dC3dt, dC4dt, dI0dt, dI1dt, dI2dt, dI3dt, dI4dt, dOdt)

In [12]:
def Inact_Protocol(max_V, DeltaV):
    Vhold = -90.0 #mV
    
    Vtest = np.linspace(Vhold,max_V,np.abs(np.int((max_V-Vhold)/DeltaV))+1)
    
    return Vtest #array of testing voltages

In [13]:
def Boltz_eq_in(x, V1_2, k):
    return 1.0/(1.0 + np.exp((x-V1_2)/k))

In [14]:
def Inact_LS_func(x, teta):

    # conductance parameters

    #EK      = 0.0    # mV
    gK_max  = 33.2     # nS 

    # Assuming no leaking
    #EL      = 0.0    # mV
    #gL_max  = 0.0    # mS 

    # Membrane capacitance
    Cm      = 1.0    # microF cm^-2

    
    Vdepo =  60.0 #mV
    
    Vhold = -90.0 #mV
    
    
    #define activation sequence protocol
    
    Vmax = 60.0 #mV
    
    increment = 10.0 #mV
    
    Vtest = Inact_Protocol(Vmax, increment)
    
    # Time discretiztion
    tini = 0.0
    
    tend = 3.00
    
    ttest = 2.00 #time at which you start record the current
     
    Npoints = 200000
    
    Points_per_sec = np.int(Npoints/tend) 
    
    # time array
    #t = np.linspace(tini,tend,Npoints)

    # prepare empty arrays
    Open_states = np.zeros((Npoints,len(Vtest)))

    max_conductance = np.zeros(len(Vtest)) 

    max_currents = np.zeros(len(Vtest)) 
    
    for i in range (0,len(Vtest)):
        gamma = np.append(teta, Vtest[i])
        
        f = lambda S,t: ode_inac_WT(S, t, gamma)
        
        r = odeint(f, S0, x)
        
        Open_states[:,i] = r[:,10]
        
        max_conductance[i] = gK_max * np.amax(r[np.int(Points_per_sec*ttest):,10]-r[np.int(Points_per_sec*ttest)-1,10])
        
        max_currents[i] = max_conductance[i] * (Vdepo - Vhold)
    
    I_Imax = (max_currents)/(np.amax(max_currents))
    
    return I_Imax

In [15]:
def residual_Inact(p):
    
    #define activation sequence protocol
    
    Vmax = 60.0 #mV
    
    increment = 10.0 #mV
    
    Vtest = Inact_Protocol(Vmax, increment)
    
    #experimental data for hemiactivation and slope
    
    exp_Vhemi_inact = -49.60 #mV
    
    exp_k_inact = 5.10 #mV
    
    #evaluation of experimental points trough the declared Boltzmann fit
    
    exp_data = Boltz_eq_in(Vtest,exp_Vhemi_inact,exp_k_inact)
    
    # Time discretiztion
    tini = 0.0
    
    tend = 3.00
    
    ttest = 2.00 #time at which you start record the current
     
    Npoints = 200000
    
    Points_per_sec = np.int(Npoints/tend) 
    
    # time array
    t = np.linspace(tini,tend,Npoints)
    
    # here data from perfect fit
    sq_err_inact = np.sum(np.subtract(exp_data,Inact_LS_func(t,p))**2)
   
    #here experimental data imported with Pandas in initial section
    #sq_err_act = np.sum(np.subtract(inact_data_WT,Inact_LS_func(t,p))**2)
    
    inact_cost_func = (1.0/len(Vtest)) * sq_err_inact
    
    return inact_cost_func

### Debugger inactivation

In [96]:
# Main 

from scipy.optimize import minimize

# Guess of initial conditions for the states 
C0_0=0.4390
C1_0=0.2588
C2_0=0.0572
C3_0=0.0056
C4_0=0.0002
I0_0=0.0128
I1_0=0.0553
I2_0=0.0894
I3_0=0.0642
I4_0=0.0172
O_0=0.0001

# Pack up the initial conditions:

S0 = [C0_0, C1_0, C2_0, C3_0, C4_0, I0_0, I1_0, I2_0, I3_0, I4_0, O_0]

# guess of the transition rates and other ODEs params (start from Lin Model)

guess = np.array([690.87, 0.252, 45.32, 2.22, 326.15, 0.369, 65.48, 0.28, 65.0, 0.21, 0.32])

# set boundaries for parameters lsq fitting based on physical reasoning

boundaries = ((400.0,1000.0),(0.0,1.0),(0.0,100.0),(0.0,10.0),(0.0,1000.0),(0.0,1.0),(0.0,100.0),(0.0,1.0),(0.0,100.0),(0.0,1.0),(0,0.5))

# Run the optimization for the cost function 

#result_lm = least_squares(residual_Act, guess, method='lm')

#result = minimize(residual_Act, guess, bounds=boundaries, method='L-BFGS-B')

result = minimize(residual_Inact, guess, method='Nelder-Mead')

print (result)

KeyboardInterrupt: 

## Recovery Part

In [16]:
def ode_rec_WT (C, t, p):
    
    C0=C[0]
    C1=C[1]
    C2=C[2]
    C3=C[3]
    C4=C[4]
    I0=C[5]
    I1=C[6]
    I2=C[7]
    I3=C[8]
    I4=C[9]
    O=C[10]

    
    #constants
    
    T = 291.0 #K or 18 degree celsius
    e =  1.602176634 * (10**-19.0) # C
    K_B = 1.380649 * (10**-23.0) # J*K^-1
    
    exp_factor = (e/(K_B * T)) * (10**-3) 

    #Voltage sequences
    
    V = 0.0 #mV 

    if 0 <= t < 1.00:
        V=-90.0
        
    if 1.00 <= t < 2.00:
        V=60.0
        
    if 2.00 <= t <= (2.00 + p[11]):
        V=-90.0
        
    if (2.00 + p[11]) < t <= (p[11] + 3.00):
        V=60.0
 
    if t > (p[11] + 3.00):
        V=-90.0

    # wild type parameters
    # wild type parameters
#    alpha_0 = 450.0 #s^-1
#    alpha_1 = 0.23 #s^-1
    
#    beta_0 = 2.0 #s^-1
#    beta_1 = 2.2 #s^-1
    
#    k_CO_0 = 160.0 #s^-1
#    k_CO_1 = 0.27 #s^-1
    
#    k_OC_0 = 245.0 #s^-1
#    k_OC_1 = 0.33 #s^-1
    
#    k_CI = 25.0 #s^-1
#    k_IC = 0.3 #s^-1
    
#    f = 0.37

    k_CI = p[8]
    
    k_IC = p[9]

    f = p[10]
    
    #voltage dependent rate constants
    
    #alpha = alpha_0 * np.exp(alpha_1 * (V * exp_factor))
    #beta = beta_0 * np.exp(-1.0 * beta_1 * (V * exp_factor))
    #k_CO = k_CO_0 * np.exp(k_CO_1 * (V * exp_factor))
    #k_OC = k_OC_0 * np.exp(-1.0 * k_OC_1 * (V * exp_factor))
    
     
    alpha = p[0] * np.exp(p[1] * (V * exp_factor))
    beta = p[2] * np.exp(-1.0 * p[3] * (V * exp_factor))
    k_CO = p[4] * np.exp(p[5] * (V * exp_factor))
    k_OC = p[6] * np.exp(-1.0 * p[7] * (V * exp_factor))
    
    
    # ODEs
    
    dC0dt = beta * C1 + (k_IC/(f**4.0)) * I0 - (k_CI*(f**4.0) + 4.0 * alpha) * C0
    dC1dt = 4.0 * alpha * C0 + 2.0 * beta * C2 + (k_IC/(f**3.0)) * I1 - (k_CI*(f**3.0) + beta + 3.0 * alpha) * C1 
    dC2dt = 3.0 * alpha * C1 + 3.0 * beta * C3 + (k_IC/(f**2.0)) * I2 - (k_CI*(f**2.0) + 2.0 * beta + 2.0 * alpha) * C2 
    dC3dt = 2.0 * alpha * C2 + 4.0 * beta * C4 + (k_IC/f) * I3 - (k_CI*f + 3.0 * beta + 1.0 * alpha) * C3 
    dC4dt = 1.0 * alpha * C3 + k_OC * O + k_IC * I4 - (k_CI + k_CO + 4.0 * beta) * C4 
    
    dI0dt = beta * f * I1 + (k_CI*(f**4.0)) * C0 - (k_IC/(f**4.0) + 4.0 * (alpha/f)) * I0
    dI1dt = 4.0 * (alpha/f) * I0 + 2.0 * beta * f * I2 + (k_CI*(f**3.0)) * C1 - (k_IC/(f**3.0) + beta * f + 3.0 * (alpha/f)) * I1 
    dI2dt = 3.0 * (alpha/f) * I1 + 3.0 * beta * f * I3 + (k_CI*(f**2.0)) * C2 - (k_IC/(f**2.0) + 2.0 * beta * f + 2.0 * (alpha/f)) * I2 
    dI3dt = 2.0 * (alpha/f) * I2 + 4.0 * beta * f * I4 + (k_CI*f) * C3 - (k_IC/f + 3.0 * beta * f + 1.0 * (alpha/f)) * I3 
    dI4dt = 1.0 * (alpha/f) * I3 + k_CI * C4 - (k_IC + 4.0 * beta * f) * I4     
    
    dOdt = k_CO * C4 - k_OC * O
    
    return (dC0dt, dC1dt, dC2dt, dC3dt, dC4dt, dI0dt, dI1dt, dI2dt, dI3dt, dI4dt, dOdt)

In [17]:
def Rec_Protocol(max_t, Deltat):
    min_t = 0.015 #mV
    
    t_pulse = np.linspace(min_t,max_t,np.abs(np.int((max_t-min_t)/Deltat))+1)
    
    return t_pulse #array of testing prepulses

In [18]:
def Simple_exp_eq(x,tau):
    return 1.0 - np.exp(-(x - 15.0 )/ tau)

In [19]:
def Rec_LS_func(x, teta):

    # conductance parameters

    #EK      = 0.0    # mV
    gK_max  = 33.2      # nS 

    # Assuming no leaking
    #EL      = 0.0    # mV
    #gL_max  = 0.0    # mS 

    # Membrane capacitance
    Cm      = 1.0    # microF cm^-2
    
    #define activation sequence protocol
    
    tmax = 0.300 #s
    
    increment = 0.015 #s
    
    t_pulse = Rec_Protocol(tmax, increment)
    
    tini_prep = 1.00 #s
    
    tend_prep = 2.00 #s
    
    Vpulse = 60.0 #mV
    
    Vhold = -90.0 #mV
    
    # Time discretiztion
    tini = 0.0 #s
    
    tend = 4.00 #s
    
    #ttest = 2.00 #time at which you start record the current
     
    Npoints = 200000
    
    Points_per_sec = np.int(Npoints/tend) 
    
    # time array
    #t = np.linspace(tini,tend,Npoints)

    # prepare empty arrays
    Open_states = np.zeros((Npoints,len(t_pulse)))

    max_conductance = np.zeros(len(t_pulse)) 
    
    max_currents = np.zeros(len(t_pulse))

    max_conductance_prep = np.zeros(len(t_pulse))

    max_currents_prep = np.zeros(len(t_pulse))
    
    for i in range (0,len(t_pulse)):
        gamma = np.append(teta, t_pulse[i])
        
        f = lambda S,t: ode_rec_WT(S, t, gamma)
        
        r = odeint(f, S0, x)
        
        Open_states[:,i] = r[:,10]
        
        max_conductance[i] = gK_max * np.amax(r[np.int(Points_per_sec*(tend_prep+t_pulse[i])):,10])
        
        max_conductance_prep[i] = gK_max * np.amax(r[np.int(Points_per_sec*tini_prep):np.int(Points_per_sec*tend_prep),10])
    
        # Compute the current proportional to the open channel conductance and potential applied
    
        max_currents[i] = max_conductance[i] * (Vpulse - Vhold)
    
        max_currents_prep[i] = max_conductance_prep[i] * (Vpulse - Vhold) # nS * mV = pA
    
    I_Imax = np.true_divide(max_currents,max_currents_prep) 
    
    return I_Imax

In [20]:
def residual_Rec(p):
    
    #define activation sequence protocol
    
    tmax = 300.0 #ms
    
    increment = 15.0 #ms
    
    t_pulse = Rec_Protocol(tmax, increment)
    
    #experimental data for hemiactivation and slope
    
    exp_tau = 47.0 #ms
    
    
    #evaluation of experimental points trough the declared Boltzmann fit
    
    exp_data = Simple_exp_eq(t_pulse,exp_tau)
    
    # Time discretiztion
    tini = 0.0
    
    tend = 4.00
    
#    ttest = 2.00 #time at which you start record the current
     
    Npoints = 200000
    
    Points_per_sec = np.int(Npoints/tend) 
    
    # time array
    t = np.linspace(tini,tend,Npoints)
    
    sq_err_rec = np.sum(np.subtract(exp_data,Rec_LS_func(t,p))**2)
    
    #here experimental data imported with Pandas in initial section
    #sq_err_rec = np.sum(np.subtract(rec_data_S390N,Rec_LS_func(t,p))**2)
    
    
    rec_cost_func = (1.0/len(t_pulse)) * sq_err_rec
    
    return rec_cost_func

### Debugger recovery

In [102]:
# Main 

from scipy.optimize import minimize

# Guess of initial conditions for the states 
C0_0=0.4390
C1_0=0.2588
C2_0=0.0572
C3_0=0.0056
C4_0=0.0002
I0_0=0.0128
I1_0=0.0553
I2_0=0.0894
I3_0=0.0642
I4_0=0.0172
O_0=0.0001

# Pack up the initial conditions:

S0 = [C0_0, C1_0, C2_0, C3_0, C4_0, I0_0, I1_0, I2_0, I3_0, I4_0, O_0]

# guess of the transition rates and other ODEs params (start from Lin Model)

guess = np.array([690.87, 0.252, 45.32, 2.22, 326.15, 0.369, 65.48, 0.28, 65.0, 0.21, 0.32])

# set boundaries for parameters lsq fitting based on physical reasoning

boundaries = ((400.0,1000.0),(0.0,1.0),(0.0,100.0),(0.0,10.0),(0.0,1000.0),(0.0,1.0),(0.0,100.0),(0.0,1.0),(0.0,100.0),(0.0,1.0),(0,0.5))

# Run the optimization for the cost function 

#result_lm = least_squares(residual_Act, guess, method='lm')

#result = minimize(residual_Act, guess, bounds=boundaries, method='L-BFGS-B')

result = minimize(residual_Rec, guess, method='Nelder-Mead')

print (result)

KeyboardInterrupt: 

## Launch main

In [21]:
def residual_Tot(p):

    # Time discretiztion
    tini = 0.0
    
    tend = 4.00
    
#    ttest = 0.56 #time at which you start record the current
     
    Npoints = 200000
    
    Points_per_sec = np.int(Npoints/tend) 
    
    # time array
    t = np.linspace(tini,tend,Npoints)

####################### START OF THE 3 VOLTAGE PROTOCOLS ###################   

    # DEFINE ACTIVATION SEQUENCE PROTOCOL
    
    Vmax = 60.0 #mV
    
    increment = 10.0 #mV
    
    Vtest_act = Act_Protocol(Vmax, increment)
    
    #experimental data for hemiactivation and slope
    
    exp_Vhemi_act = -22.70 #mV
    
    exp_k_act = 13.00 #mV
    
    #evaluation of experimental points trough the declared Boltzmann fit
    
    exp_data_act = Boltz_eq(Vtest_act,exp_Vhemi_act,exp_k_act)
    
    # here data from perfect fit
    sq_err_act = np.sum(np.subtract(exp_data_act,Act_LS_func(t,p))**2)
    
    #here experimental data imported with Pandas in initial section
    #sq_err_act = np.sum(np.subtract(act_data_WT,Act_LS_func(t,p))**2)
    
    act_cost_func = (1.0/len(Vtest_act)) * sq_err_act
    
    
    # DEFINE INACTIVATION SEQUENCE PROTOCOL
    
#    Vmax = 60.0 #mV
    
#    increment = 10.0 #mV
    
    Vtest_inact = Inact_Protocol(Vmax, increment)
    
    #experimental data for hemiactivation and slope
    
    exp_Vhemi_inact = -49.60 #mV
    
    exp_k_inact = 5.10 #mV
    
    #evaluation of experimental points trough the declared Boltzmann fit
    
    exp_data_inact = Boltz_eq_in(Vtest_inact,exp_Vhemi_inact,exp_k_inact)
    
    # here data from perfect fit
    sq_err_inact = np.sum(np.subtract(exp_data_inact,Inact_LS_func(t,p))**2)
    
    #here experimental data imported with Pandas in initial section
    #sq_err_inact = np.sum(np.subtract(inact_data_WT,Inact_LS_func(t,p))**2)
    
    inact_cost_func = (1.0/len(Vtest_inact)) * sq_err_inact
   
   
    # DEFINE RECOVERY SEQUENCE PROTOCOL
    
    tmax = 300.0 #ms
    
    increment = 15.0 #ms
    
    t_pulse = Rec_Protocol(tmax, increment)
    
    #experimental data for recovery time
    
    exp_tau = 47.0 #ms
    
    #evaluation of experimental points trough the declared exponential fit
    
    exp_data_rec = Simple_exp_eq(t_pulse,exp_tau)
    
    # here data from perfect fit
    sq_err_rec = np.sum(np.subtract(exp_data_rec,Rec_LS_func(t,p))**2)
    
    #here experimental data imported with Pandas in initial section
    #sq_err_rec = np.sum(np.subtract(rec_data_WT,Rec_LS_func(t,p))**2)
    
    rec_cost_func = (1.0/len(t_pulse)) * sq_err_rec
    
    
    # sum of the three protocols cost function
    return (act_cost_func + inact_cost_func + rec_cost_func)

In [22]:
# Main 

from scipy.optimize import minimize

# Guess of initial conditions for the states 
C0_0 = 43.90
C1_0 = 25.88
C2_0 = 5.72
C3_0 = 0.56
C4_0 = 0.02
I0_0 = 1.28
I1_0 = 5.53
I2_0 = 8.94
I3_0 = 6.42
I4_0 = 1.72
O_0  = 0.0

# Pack up the initial conditions:

S0 = [C0_0, C1_0, C2_0, C3_0, C4_0, I0_0, I1_0, I2_0, I3_0, I4_0, O_0]

# guess of the transition rates and other ODEs params (start from Lin Model and some trial)

guess = np.array([3350.87, 0.302, 105.32, 2.02, 676.15, 0.269, 85.48, 0.009, 130.00, 0.41, 0.44])

    
# set boundaries for parameters lsq fitting based on physical reasoning

#boundaries = ((400.0,1000.0),(0.0,1.0),(0.0,100.0),(0.0,10.0),(0.0,1000.0),(0.0,1.0),(0.0,100.0),(0.0,1.0),(0.0,100.0),(0.0,1.0),(0,0.5))

# Run the optimization for the cost function 

result = minimize(residual_Tot, guess, method='Nelder-Mead', options={'maxiter':10000,'maxfev':100000})

print (result)


/usr/local/lib/python2.7/dist-packages/scipy/integrate/odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


 final_simplex: (array([[2.89831622e+03, 3.85937138e-01, 1.52101853e+02, 2.48747741e+00,
        6.17021508e+02, 1.07132153e-01, 1.12150455e+02, 8.50035103e-03,
        1.06594552e+02, 3.48794284e-01, 5.89104092e-01],
       [2.89831622e+03, 3.85937138e-01, 1.52101853e+02, 2.48747741e+00,
        6.17021508e+02, 1.07132153e-01, 1.12150455e+02, 8.50035103e-03,
        1.06594552e+02, 3.48794284e-01, 5.89104092e-01],
       [2.89831622e+03, 3.85937138e-01, 1.52101853e+02, 2.48747741e+00,
        6.17021508e+02, 1.07132153e-01, 1.12150455e+02, 8.50035103e-03,
        1.06594552e+02, 3.48794284e-01, 5.89104092e-01],
       [2.89831622e+03, 3.85937138e-01, 1.52101853e+02, 2.48747741e+00,
        6.17021508e+02, 1.07132153e-01, 1.12150455e+02, 8.50035103e-03,
        1.06594552e+02, 3.48794284e-01, 5.89104092e-01],
       [2.89831622e+03, 3.85937138e-01, 1.52101853e+02, 2.48747741e+00,
        6.17021508e+02, 1.07132153e-01, 1.12150455e+02, 8.50035103e-03,
        1.06594552e+02, 3.48794284e